In [45]:
import pandas as pd  # module for working with data sets (need for working inner classes and modules)
import traceback
import datetime
import subprocess
import warnings
import numpy as np
import os  # module for working with operating system catalog structure
import openpyxl  # module for working with Excel files
import time  # module for working with date and time
import pyodbc  # module for working with databases
import win32com.client  # Module for generating MS access data base

from datetime import datetime  # Module for working with date and time data
from win32com.client import Dispatch  # Module for generating MS access data base
from tkinter.filedialog import askopenfilename  # Module for open file with win gui

# Ignoring pandas version errors
warnings.simplefilter(action='ignore', category=(FutureWarning, UserWarning))

# show an "Open" dialog box and return the path to the selected file
filename_comp = askopenfilename(title="Select file for compare", filetypes=[("excel files", "*.xlsx")])
database_root = askopenfilename(title="Select database", filetypes=[('*.mdb', '*.accdb')]).replace('/', '\\')
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    fr'DBQ={database_root};'
    )

table = input("Input table's name : ")
with pyodbc.connect(conn_str) as conn:
    # Создаем курсор для выполнения запросов
    query = f'''SELECT * FROM {table}'''
    new_df = pd.read_sql(query, conn)

columns = ['Коды работ по выпуску РД', 'Наименование объекта/комплекта РД']

inf_columns = ['Наименование объекта/комплекта РД',
               'Коды работ по выпуску РД', 
               'Пакет РД', 
               'Код KKS документа',
               'Статус Заказчика', 
               'Текущая ревизия', 
               'Статус текущей ревизии',
               'Дата выпуска РД по договору подрядчика',
               'Дата выпуска РД по графику с Заказчиком',
               'Дата статуса Заказчика',
               'Ожидаемая дата выдачи РД в производство', 
               'Письма',
               'Источник информации', 
               'Разработчики РД (актуальные)', 
               'Статус РД в 1С'
               ]
changed_cols = ['Коды работ по выпуску РД',
        'Наименование объекта/комплекта РД',
       'Код KKS документа',
       'Пакет РД',
       'Статус Заказчика', 
       'Текущая ревизия', 
       'Статус текущей ревизии',
       'Дата выпуска РД по договору подрядчика',
       'Дата выпуска РД по графику с Заказчиком', 'Дата статуса Заказчика',
       'Ожидаемая дата выдачи РД в производство',
       'Разработчики РД (актуальные)']

base_columns = ['Система',
                'Наименование объекта/комплекта РД',
                'Коды работ по выпуску РД',
                'Тип',
                'Пакет РД',	'Код KKS документа',
                'Статус Заказчика',
                'Текущая ревизия',
                'Статус текущей ревизии',
                'Дата выпуска РД по договору подрядчика',
                'Дата выпуска РД по графику с Заказчиком',
                'Дата статуса Заказчика',	
                'Ожидаемая дата выдачи РД в производство',	
                'Письма',	
                'Источник информации',	
                'Разработчики РД (актуальные)',	
                'Объект',	
                'WBS',	
                'КС',	
                'Примечания']

# read Excel files with current and new data
print('Read excel files with current and new data')
base_df = pd.read_excel(filename_comp)
new_df.columns = base_columns

#  Clear the data in both dataframe
print('Clear the empty rows in both dataframes')
base_df = base_df.dropna(subset=['Коды работ по выпуску РД'])
base_df['Разработчики РД (актуальные)'] = base_df['Разработчики РД (актуальные)'].apply(
    lambda row: base_df['Разработчик РД'] if row is None else row
    )

# Removing unnecessary data
print('Clear the unnecessary data in base dataframe')
base_df = base_df.loc[(~base_df['Коды работ по выпуску РД'].str.contains('.C.'))]
base_df = base_df.loc[((~base_df['Код KKS документа'].isin(['.KZ.', '.EK.', '.TZ.', '.KM.', '.GR.'])))]

#  Making copy of original dataframes
print('Making copy of original dataframes')
base_df_copy = base_df.copy()
new_df_copy = new_df.copy()



Read excel files with current and new data
Clear the empty rows in both dataframes
Clear the unnecessary data in base dataframe
Making copy of original dataframes


In [49]:
#  Merging two dataframes dddd
print('Merging two dataframes')
missed_code = (new_df.merge(base_df,
                           how='outer',
                           on=['Коды работ по выпуску РД'],
                           suffixes=['', '_new'], 
                           indicator=True))

missed = missed_code[missed_code['_merge'] == 'left_only'].reset_index()[columns]
cods = base_df['Коды работ по выпуску РД'].apply(lambda row: row if row not in new_df['Коды работ по выпуску РД'].unique() else None)

Merging two dataframes


0                     None
1                     None
2                     None
3       00UGM.E.CY0.D-1000
4       00UGM.E.B00.D-1030
               ...        
3895    10UKC.R.CF0.D-1060
3896    02UYX.R.CF0.D-1060
3897    02UYX.R.CF0.D-1050
3898    02UYX.R.CFU.D-1040
3899    02UYX.R.CF0.D-1020
Name: Коды работ по выпуску РД, Length: 3900, dtype: object

In [51]:
cods[~cods.isna()]

3       00UGM.E.CY0.D-1000
4       00UGM.E.B00.D-1030
5       00UGM.E.BJ0.D-1000
6       00UGV.E.B00.D-1000
7       00UGW.E.CY0.D-1000
               ...        
3895    10UKC.R.CF0.D-1060
3896    02UYX.R.CF0.D-1060
3897    02UYX.R.CF0.D-1050
3898    02UYX.R.CFU.D-1040
3899    02UYX.R.CF0.D-1020
Name: Коды работ по выпуску РД, Length: 2263, dtype: object